# MMA Offshore’s PE

***This project investigates the P/E ratio of an ASX-listed company to gain insight into the approach used by asset managers.***

***
### Section 1
> *Create a simple profit and loss account (using the same line-items as the company does) in Excel for the last 3 full financial years. Focus on continuing operations (we will assume the company is honest about what is a ‘one-off’ item of expense or income). Use Excel formulas to calculate sub-totals and totals.*

Financials data has been collected from annual reports retrieved from the company website: https://www.mmaoffshore.com/investor-centre/financial-reports

Profit and loss accounts for continuing operations has stored in a CSV file: https://github.com/lei-per/MMA/blob/master/financials.csv

The script below reads the data from the CSV file into a dataframe in R.

In [1]:
# Read data from CSV file
financials <- read.csv("financials.csv", row.names = 1, header = TRUE)
financials

,FY17,FY18,FY19
revenue,221766,200444,239259
invest_income,133,463,1278
other,-14960,87,-556
vessel_expenses,-241636,-206484,-238951
admin_expenses,-7377,-7092,-7402
impairment,-287542,8407,-10361
finance_costs,-26444,-23201,-19146
income_tax,1729,-533,-1494


Now that the data has been imported, we can calculate the total profit or loss for each financial year (`profit_loss_total`).

The weighted average number of ordinary shares used in the calculation of earnings per share (`ordinary_shares`) has been used to calculate the earnings per share (`earnings_per_share`). The earnings per share figures calculated match those in the company's annual reports.


The same calculations have also been repeated in Excel: https://github.com/lei-per/MMA/blob/master/financials.xlsx

The results show that the earnings per share are negative in each year. This makes sense as the company has reported a net loss each year.


In [2]:
# Calculate profit or loss totals for each financial year
profit_loss_total <- colSums(financials[1:8, ])

# Set the number of ordinary shares used in the calculation of earnings per share
ordinary_shares <- c(402771, 678468, 858077)
names(ordinary_shares) <- c("FY17", "FY18", "FY19")

# Calculate earnings per share (in cents per share)
earnings_per_share <- round(profit_loss_total / ordinary_shares * 100, 3)

# Summarise results of the analysis in a dataframe
analysis <- data.frame(profit_loss_total, ordinary_shares, earnings_per_share)
analysis

,profit_loss_total,ordinary_shares,earnings_per_share
FY17,-354331,402771,-87.973
FY18,-27909,678468,-4.114
FY19,-37373,858077,-4.355


***
### Section 2
> *Add a single financial year forecast and assume last years revenue growth continues and all other income and expenses grow by 5%. Show your forecast profit for MRM on this basis.*

The script below forecasts the total profit or loss for the 2020 financial year (`profit_loss_total_FY20_forecast`).

Based on the assumptions above, a loss of \\$4.87m is forecast for MRM in the 2020 financial year.

In [3]:
# Calculate a forecast for the 2020 financial year by assuming all income and expenses grow by 5%
FY20_forecast <- round(1.05 * financials[1:8, "FY19"])
names(FY20_forecast) <- rownames(financials[1:8, ])

# Update the revenue forecast using last years revenue growth rate
growth_rate <- financials["revenue", "FY19"] / financials["revenue", "FY18"]
FY20_forecast["revenue"] <- round(growth_rate * financials["revenue", "FY19"])
data.frame(FY20_forecast)

# Calculate profit or loss total for the 2020 financial year forecast
profit_loss_total_FY20_forecast <- sum(FY20_forecast)
names(profit_loss_total_FY20_forecast) <- "profit_loss_total_FY20_forecast"
profit_loss_total_FY20_forecast

,FY20_forecast
revenue,285590
invest_income,1342
other,-584
vessel_expenses,-250899
admin_expenses,-7772
impairment,-10879
finance_costs,-20103
income_tax,-1569


profit_loss_total_FY20_forecast 
                          -4874

***
### Section 3
> *Find historical share prices anywhere online for MRM and work out what approximate Price-to-Earnings ratio (PE) the company was trading at, 1 month after it reported for each of the last 2 years. Do you think this PE is a good way of valuing this company? (If not, can you suggest a different approach).*

The `quantmod` package in R has been used to retrieve stock data from yahoo finance.

In [6]:
# Load the quantmod package
library("quantmod")

In [7]:
# Use getSymbols from the quantmod package to get stock data from yahoo finance
MRM.AX <- getSymbols("MRM.AX", src = "yahoo", auto.assign = FALSE)

# Display the first and last three rows of the stock data
data.frame(head(MRM.AX, 3))
data.frame(tail(MRM.AX, 3))

,MRM.AX.Open,MRM.AX.High,MRM.AX.Low,MRM.AX.Close,MRM.AX.Volume,MRM.AX.Adjusted
2007-01-02,0.86992,0.87436,0.84329,0.86992,79427,0.616371
2007-01-03,0.87436,0.87436,0.83442,0.86992,133652,0.616371
2007-01-04,0.86105,0.87436,0.84329,0.86992,93353,0.616371


,MRM.AX.Open,MRM.AX.High,MRM.AX.Low,MRM.AX.Close,MRM.AX.Volume,MRM.AX.Adjusted
2019-12-12,0.170,0.17,0.160,0.17,589704,0.17
2019-12-13,0.165,0.17,0.165,0.17,251122,0.17
2019-12-16,0.170,0.17,0.170,0.17,199546,0.17


Preliminary financial reports were released on 28th August 2017, 27th August 2018 and 26th August 2019. Share prices one month after each report have been used to calculate the price-earnings ratio (`pe_ratio`). 

In [8]:
# Subset stock data to select prices (in cents) one month after each report was released
price_per_share <- as.vector(MRM.AX[c("2017-09-28", "2018-09-27", "2019-09-26"), "MRM.AX.Adjusted"] * 100)

# Calculate PE ratios
pe_ratio <- round(price_per_share / earnings_per_share, 3)

# Add new columns to the analysis dataframe with the results
analysis$price_per_share <- price_per_share
analysis$pe_ratio <- pe_ratio
analysis

,profit_loss_total,ordinary_shares,earnings_per_share,price_per_share,pe_ratio
FY17,-354331,402771,-87.973,17.585,-0.200
FY18,-27909,678468,-4.114,24.000,-5.834
FY19,-37373,858077,-4.355,19.000,-4.363


The results show the PE ratio is negative each year, at a multiple of -5.003 for FY17, then increasing substantially to -0.171 for FY18 and then decreasing slightly to -0.229 for FY19. In simplified terms, PE ratio can be thought of as the cost at which investors may obtain a given rate of corporate earnings. For this reason, PE ratio is commonly used to determine whether a company is overvalued (high PE) or undervalued (low PE) at its current share price. However, this relationship no longer holds when PE ratio is negative. For example, when the share price increased from 24 cents in FY18 to 19 cents in FY19, this decrease in the share price resulted in an *increase* to the PE ratio. For this reason, when the PE ratio is negative it is no longer a useful measure of relative value.

One alternative would be to calculate a forward PE ratio using forecast earnings. However, the forward PE ratio would not offer any advantage in this case, as the forecast for FY20 earnings is also negative (forecasted loss of $4.87m).

Another alternative is to use the Enterprise Value (EV) / EBITDA ratio. The EV/EBITDA ratio may be thought of as the cost to acquire the entire business, as a multiple of its operating profit. Similar to the PE ratio, this ratio gives a measure of relative value and a low EV/EBITDA ratio may indicate that a company is undervalued.  The EV/EBITDA ratio is often used as it allows for a comparison of company value independent of capital structure.

To calculate EV/EBITDA requires additional information from the financial reports to be loaded into R. This has been done in the Appendix section.

***
### Section 4
> *Create a working/tested R program to load monthly share prices for MMA offshore for the 2018 calendar year. Source the prices online and get them into R in whatever way you this is most useful.*

Stock data has already been loaded for MMA Offshore using the `quantmod` package. The `timeDate` package has been used below to select prices at the end of each month for the 2018 calendar year.  

In [9]:
# Load the timeDate package
library("timeDate")

In [10]:
# Subset stock data to select prices for the 2018 calendar year (in cents)
days_CY18 <- seq(as.Date("2018-01-01"), as.Date("2018-12-31"), by="days")
prices_CY18 <- MRM.AX[days_CY18, "MRM.AX.Adjusted"] * 100

# Use apply.monthly from the timeDate package to select prices at the end of each month
prices_CY18_monthly <- apply.monthly(prices_CY18, head, 1)
names(prices_CY18_monthly) <- "price_monthly"
data.frame(prices_CY18_monthly)

,price_monthly
2018-01-31,28.5
2018-02-28,27.0
2018-03-29,23.0
2018-04-30,24.0
2018-05-31,22.5
2018-06-29,23.0
2018-07-31,24.5
2018-08-31,27.0
2018-09-28,22.0
2018-10-31,25.0


***
### Section 5
> *Load the FY2018 eps into R and calculate a PE for each month of Calendar year 2018. Share any thoughts you have about why it might change throughout the year.*

The script below calculates the PE ratio for each month of the 2018 calendar year using the FY18 earnings per share. 

In [11]:
# Calculate monthly PE ratio
pe_ratio_monthly <- round(prices_CY18_monthly / analysis["FY18", "earnings_per_share"], 3)
names(pe_ratio_monthly) <- "pe_ratio_monthly"

# Summarise results in the analysis_monthly dataframe
analysis_monthly <- data.frame(prices_CY18_monthly, pe_ratio_monthly)
analysis_monthly

,price_monthly,pe_ratio_monthly
2018-01-31,28.5,-6.928
2018-02-28,27.0,-6.563
2018-03-29,23.0,-5.591
2018-04-30,24.0,-5.834
2018-05-31,22.5,-5.469
2018-06-29,23.0,-5.591
2018-07-31,24.5,-5.955
2018-08-31,27.0,-6.563
2018-09-28,22.0,-5.348
2018-10-31,25.0,-6.077


The results show that as the monthly share price has decreased from 28.5 cents down to 20.0 cents throughout the year, the PE ratio has increased from -6.944 to -4.854 over the same period. This is due the assumption that earnings per share remains constant. Typically, as the share price decreases so does the 'multiple' of the PE ratio, indicating that the company is trading at a lower value relative to its earnings. However, as the PE ratio is negative it actually represents the price as a 'multiple' of the company's losses, which is less meaningful for valuation. 

The EV/EBITDA ratio has been calculated for each month in the Appendix section as a comparison.

***
### Section 6
> **`a.`** *Doing some research online, what reasons (in order of importance) would you give to explain Forager’s interest in MRM?*

The two key reasons for Forager's interest in MMA Offshore as an investment are:

1. Forager's focus on value investing an investing in companies trading at a discount to intrinsic value.

    At the time of Forager's initial investment in early 2015, MMA had an enterprise value of \\$700m and \\$1.3bn of tangible assets. As a value fund manager Forager takes the view the although security prices vary enormously over the course of a year but a business’s revenue, profit and cash flow rarely change as much as its security price. Forager's investment strategy is to identify and invest in securities where the estimated intrinsic value is substantially higher than the security price
 
  
2. Forager's contrarian view on a long term recovery in oil prices.

    Forager's analysis of the macro economy has identified that the world is facing the prospect of a significant shortage of oil over the next decade due to a lack of investment as oil prices have declined. Forager has identified that in the Australian market it is easier to gain exposure to a recovery in the oil price through oil services companies such as MMA Offshore.

Sources:  
https://foragerfunds.com/wp-content/uploads/2019/04/PDS-FASF_NOV16_FINAL.pdf  
https://foragerfunds.com/news/the-looming-oil-shortage/  
https://foragerfunds.com/news/investing-oil-price-recovery/


> **`b.`** *Can you highlight any specific risks that MRM poses for Forager as an investment?*

Forager's investment in MMA Offshore presents a number of risks that Forager will look to monitor closely:

 1. Oil price volatility.
     
    Despite a recovery in oil prices, oil services companies may benefit more from oil price stability. Price stability would lead to confidence and greater spending on longer term projects likely to benefit services providers. This poses a risk for Forager, as although they may be correct in their forecast of an oil price recovery their investments may not benefit until oil prices maintain stability.  
     
     
 2. Debt burden.
     
    Forager will look to monitor MMA Offshore's levels of debt very closely. If the company maintains high levels of debt, this will diminish the company's ability to wait for an oil price recovery and effectively capitalise on such a recovery.
     
     
 3. Liquidity risk.
     
    Forager Funds is a substantial shareholder, owning 5.26\% of issued capital in MMA Offshore. If Forager decides to dispose of the holding there may be substantial slippage costs and it may be difficult for Forager to dispose of the holding in a discreet or timely manner due to lack of liquidity.
     
     
 4. Concentration risk.
     
    Forager's Australian Shares Fund had a 6.0% allocation of its portfolio to MMA Offshore, as of the September 2019 quarterly report.
          
    
 5. Foreign currency exposure.
    
    MMA Offshore is primarily exposed to US dollar exchange rate. The sensitivity analysis in the 2019 annual report shows that if the  Australian dollar strengthens 10\% against the US dollar, this would result in a \\$0.96m loss in revenue.
     
     
   
Sources:  
https://foragerfunds.com/news/on-the-oil-rollercoaster/  
https://www.mmaoffshore.com/theme/mmaoffshorecomau/assets/public/File/asx/190920_-_2019_Annual_Report.pdf  
https://foragerfunds.com/wp-content/uploads/2019/10/QR-ForagerFunds_SEP19.pdf

***
### Appendix

The script below adds depreciation to the financials data to calculate EBITDA (Earnings Before Interest, Taxes, Depreciation and Amortization) for the past three financial years.

`ebitda = profit_loss_total - finance_costs - income_tax - impairment + depreciation`

It's worth noting that the EBITDA figures calculated differ from those reported in the company's annual reports. This may be due to a difference in accounting methods, as the figures calculated below match those reported by Morningstar.

In [12]:
# Add depreciation data to the financials data
depreciation <- c(45541, 31903, 35319)
financials["depreciation",] <- depreciation

# Calculate EBITDA 
ebitda <- colSums(financials[1:9, ]) - colSums(financials[6:8,])
data.frame(ebitda)

,ebitda
FY17,3467
FY18,19321
FY19,28947


The script below adds cash equivalents and non current liabilities to the financials data to calculate Enterprise Value (EV).

`enterprise_value = ordinary_shares * price_per_share + noncurrent_liabilities - cash_equivalents`  

The EV/EBITDA ratio is then calculated for the past three financial years.

In [13]:
# Add cash equivalents and non current liabilities to the financials data
cash_equivalents <- c(28757, 69648, 70155)
noncurrent_liabilities <- c(323929, 265215, 268255)
financials["cash_equivalents",] <- cash_equivalents
financials["noncurrent_liabilities",] <- noncurrent_liabilities
financials

# Calculate Enterprise Value 
enterprise_value <- round(ordinary_shares * price_per_share / 100 + noncurrent_liabilities - cash_equivalents)

# Calculate EV/EBITDA
ev_ebitda_ratio <- round(enterprise_value / ebitda, 3)

# Add new columns to the analysis dataframe with the results
analysis$ebitda <- ebitda
analysis$enterprise_value <- enterprise_value
analysis$ev_ebitda_ratio <- ev_ebitda_ratio
analysis

,FY17,FY18,FY19
revenue,221766,200444,239259
invest_income,133,463,1278
other,-14960,87,-556
vessel_expenses,-241636,-206484,-238951
admin_expenses,-7377,-7092,-7402
impairment,-287542,8407,-10361
finance_costs,-26444,-23201,-19146
income_tax,1729,-533,-1494
depreciation,45541,31903,35319
cash_equivalents,28757,69648,70155


,profit_loss_total,ordinary_shares,earnings_per_share,price_per_share,pe_ratio,ebitda,enterprise_value,ev_ebitda_ratio
FY17,-354331,402771,-87.973,17.585,-0.200,3467,365999,105.566
FY18,-27909,678468,-4.114,24.000,-5.834,19321,358399,18.550
FY19,-37373,858077,-4.355,19.000,-4.363,28947,361135,12.476


The script below calculates the EV/EBITDA ratio for each month of the 2018 calendar year using the FY18 financials data.

In [14]:
# Calculate monthly EV / EBITDA ratios
enterprise_value_monthly <- analysis["FY18", "ordinary_shares"] * prices_CY18_monthly / 100 + 
    financials["noncurrent_liabilities", "FY18"] - financials["cash_equivalents", "FY18"]
ev_ebitda_ratio_monthly <- round(enterprise_value_monthly / analysis["FY18", "ebitda"], 3)
names(ev_ebitda_ratio_monthly) <- "ev_ebitda_ratio"

# Add new columns to the analysis_monthly dataframe with the results
analysis_monthly$ev_ebitda_ratio <- ev_ebitda_ratio_monthly
analysis_monthly

,price_monthly,pe_ratio_monthly,ev_ebitda_ratio
2018-01-31,28.5,-6.928,20.130
2018-02-28,27.0,-6.563,19.603
2018-03-29,23.0,-5.591,18.199
2018-04-30,24.0,-5.834,18.550
2018-05-31,22.5,-5.469,18.023
2018-06-29,23.0,-5.591,18.199
2018-07-31,24.5,-5.955,18.725
2018-08-31,27.0,-6.563,19.603
2018-09-28,22.0,-5.348,17.847
2018-10-31,25.0,-6.077,18.901


Unlike the PE ratio, the results show that the EV/EBITDA ratio moves in tandem with the monthly share price. A higher EV/EBITDA ratio indicates a higher valuation for the company relative to its operating cash flow.